# Surface-based segregation

## Introduction
#todo


In [11]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

from spatial_segregation.spatial_segregation.surface_based import surface_dissim


ModuleNotFoundError: No module named 'spatial_segregation.spatial_segregation'


## Aspatial indices
#todo

## Kernels
#todo

## Synthetic data

Testing the measurements using synthetic data gives some idea about the nature of measurement.
O'Sullivan and Wong use synthetic data representing four different scenarios.
We will plot inhabitants using colored squares and the area of interest using a red boundary.


In [ ]:
corners = [
    (9.5, 9.5),
    (9.5, 29.5),
    (29.5, 29.5),
    (29.5, 9.5),
    (9.5, 9.5),
]
area_of_interest = Polygon(corners)
x = np.arange(0, 40)
y = np.arange(0, 40)
xx, yy = np.meshgrid(x, y)
points = [Point(x, y) for (x, y) in zip(xx.ravel(), yy.ravel())]
geometry = gpd.GeoSeries(pd.Series(points))



i) 1-km checkerboard


In [ ]:
black = [1000 if p.x % 2 == p.y % 2  else 0 for p in geometry]
white = [1000 if p.x % 2 != p.y % 2  else 0 for p in geometry]
df1 = gpd.GeoDataFrame({'geometry': geometry, 'white': white, 'black': black})

df1.plot(column='black', markersize=18, cmap='binary', marker='s')
plt.plot(*area_of_interest.exterior.xy, lw=3, color='red')


ii) 5-km checkerboard


In [ ]:
black = [1000 if p.x // 5 % 2 == p.y // 5 % 2  else 0 for p in geometry]
white = [1000 if p.x // 5 % 2 != p.y // 5 % 2  else 0 for p in geometry]
df2 = gpd.GeoDataFrame({'geometry': geometry, 'white': white, 'black': black})

df2.plot(column='black', markersize=18, cmap='binary', marker='s')
plt.plot(*area_of_interest.exterior.xy, lw=3, color='red')


iii) East-west segregated


In [ ]:

black = [1000 if p.x < 20 else 0 for p in geometry]
white = [1000 if p.x > 20 else 0 for p in geometry]
df3 = gpd.GeoDataFrame({'geometry': geometry, 'white': white, 'black': black})

df3.plot(column='black', markersize=18, cmap='binary', marker='s')
plt.plot(*area_of_interest.exterior.xy, lw=3, color='red')



iv) "Ghetto"


In [ ]:

ghetto = Point(17.5, 27.5).buffer(10)
black = [1000 if ghetto.contains(p) else 0 for p in geometry]
white = [1000 if not ghetto.contains(p) else 0 for p in geometry]
df4 = gpd.GeoDataFrame({'geometry': geometry, 'white': white, 'black': black})

df4.plot(column='black', markersize=18, cmap='binary', marker='s')
plt.plot(*area_of_interest.exterior.xy, lw=3, color='red')


In [ ]:

for df in df1, df2, df3, df4:
    s, _ = surface_dissim(
        data,
        function='quartic',
        bandwidth=2.5,
        group_1_pop_var='black',
        group_2_pop_var='white',
    )
    print(s)



Only using area of interest:


In [ ]:

for df in df1, df2, df3, df4:
    data = df.cx[9.5:29.5, 9.5:29.5]
    s, _ = surface_dissim(
        df,
        function='quartic',
        bandwidth=2.5,
        group_1_pop_var='black',
        group_2_pop_var='white',
    )
    print(s)




Results

|Number |Description          | S (global) | S (area of interest) |
|-------|---------------------|------------|----------------------|
|i      |1-km checkerboard    |
|ii     |5-km checkerboard    |
|iii    |East-west segregated |
|iv     |"Ghetto"             |

## References

* O'Sullivan, D. and Wong, D. W. (2007), A Surface‐Based Approach to Measuring Spatial Segregation.
Geographical Analysis, 39: 147-168. doi:10.1111/j.1538-4632.2007.00699.x

